In [138]:
import pandas as pd

In [139]:
# load titanic data set
import pandas as pd
path = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/titanic.csv'
titanic = pd.read_csv(path,index_col='PassengerId')
titanic.head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [140]:
# lots of imports
# preprocessing
from sklearn.impute import SimpleImputer # impute missing values (nans)
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PolynomialFeatures


# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

# classification model
from sklearn.neighbors import KNeighborsClassifier

# grid search
from sklearn.model_selection import GridSearchCV, train_test_split

# optional: pipeline visual
from sklearn import set_config
set_config(display='diagram')

## Classification pipeline
* 1. Data of some sort
* 2. Add new features (numerical or categorial features)
* 3. For numerical features: impute nans, scale
* 3. For categorial features: impute + oneHotEncoder
* 4. For either category, use polynomial features
* 5. Pass polynomial features into classification model (such as knn)
* 6. Analyze predictions

## More stuff
* Numberical features: class, age, fare, family size
* Categorical features: sex, embarked, title

* Numerical feature pipeline: impute missing values --> scale
* Categorial feature pipeline: impute missing values --> encode (onehot encoding)

## Adding new features


In [141]:
# title
titanic.Name[1].split(',')[1].split(' ')[1]

'Mr.'

In [142]:
# apply functions by using lambda notation
title = titanic.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())

In [143]:
title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [144]:
# limit titles to top 6, put it in a list
most_common_titles = title.value_counts().head(6).index.to_list()
most_common_titles

['Mr', 'Miss', 'Mrs', 'Master', 'Dr', 'Rev']

In [145]:
# replace less common titles with 'Other'
title[~title.isin(most_common_titles)] = 'Other'
title.value_counts()

Mr        517
Miss      182
Mrs       125
Master     40
Other      14
Dr          7
Rev         6
Name: Name, dtype: int64

In [146]:
# family size: SibSp (siblings) + Parch (parents + children) + 1 (person themself)
family_size = titanic.SibSp+titanic.Parch+1
family_size.value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
dtype: int64

In [147]:
def get_title(dataframe):
    df = dataframe.copy()
    df['Title'] = df.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
    df.loc[~df.Title.isin(most_common_titles), 'Title'] = 'Other'
    return df


In [148]:
get_title(titanic)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Miss


In [149]:
def get_family_size(dataframe):
    df = dataframe.copy()
    df['Family_Size'] = df.SibSp + df.Parch + 1
    return df

In [150]:
get_family_size(titanic)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_Size
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,4


In [151]:
title_processor = FunctionTransformer(get_title)
family_size_processor = FunctionTransformer(get_family_size)

Pipeline applies sequentiall a list of transformations

In [152]:
# Setup numerical scaler
num_features = ['Pclass', 'Age', 'Fare', 'Family_size'] # numerical features
#setup num pipeline
num_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # strategy = replace missing values with mean or median
    ('scaler', MinMaxScaler())
])
num_processor


Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler())])

In [153]:
## Setup categorical pipeline
cat_features = ['Sex', 'Embarked', 'Title']
cat_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # use most frequent for missing categorical values 
    ('encoder', OneHotEncoder())
])
cat_processor

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder())])

In [154]:
# for both pipelines, we use a column transformer. Pipeline 1 = numerical features. Pipeline 2 = categorical features
features_processor = ColumnTransformer(transformers=[
    ('numerical processor', num_processor, num_features),
    ('categorical processor', cat_processor, cat_features)
],remainder='drop') # drop unused features. Name, SipSp, Parch, Cabin, Ticker
features_processor

ColumnTransformer(transformers=[('numerical processor',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', MinMaxScaler())]),
                                 ['Pclass', 'Age', 'Fare', 'Family_size']),
                                ('categorical processor',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 ['Sex', 'Embarked', 'Title'])])

In [155]:
# classifier
knn = KNeighborsClassifier()

In [156]:
# classification pipeline
pipe = Pipeline(steps=[
    ('get title', title_processor),
    ('get family size', family_size_processor),
    ('feature processor', features_processor),
    ('polynomial_features', PolynomialFeatures), # can set degree? either use 1 or 2
    ('classifier', knn),
])
pipe

TypeError: get_params() missing 1 required positional argument: 'self'

TypeError: get_params() missing 1 required positional argument: 'self'

Pipeline(steps=[('get title',
                 FunctionTransformer(func=<function get_title at 0x000002A282F95A60>)),
                ('get family size',
                 FunctionTransformer(func=<function get_family_size at 0x000002A283902820>)),
                ('feature processor',
                 ColumnTransformer(transformers=[('numerical processor',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Pclass', 'Age', 'Fare',
                                                   'Family_size']),
                                                 ('categorical processor',
                                                  Pipeline(steps=[('imputer',
                  

In [157]:
# grid search (degree of poly features, for knn: n_neighbors, weights)
param_dic = {'classifier__n_neighbors': list(range(1,30)),
            'classifier__weights':['uniform', 'distance'],
             'polynomial_features__degree': [1,2],
            }
grid = GridSearchCV(pipe, param_dic, cv=5, scoring='accuracy', n_jobs=-1, verbose=1) # cv should be 5-10

In [158]:
# feature matrix X and target vector y
X = titanic.drop('Survived', axis=1)
y = titanic['Survived']
# X_train, y_train, X_test, y_test

In [159]:
grid.fit(X,y)

Fitting 5 folds for each of 116 candidates, totalling 580 fits


TypeError: get_params() missing 1 required positional argument: 'self'

In [ ]:
# get best hyper parameters
grid.best_params_

In [ ]:
# Would I have survived the titanic distaster?
passenger = {
            'Pclass': [1],
            'Name': ['Derrick-Nemetchek, Dr. something'],
            'Sex': ['male'],
            'Age': [21],
            'Sibsp': [5],
            'Parch': [2],
            'Ticket': ['doesnt matter'],
            'Fare': [100],
            'Cabin': ['also doesnt matter'],
            'Embarked': ['S']
            }
passenger = pd.DataFrame(passenger)
passenger

In [ ]:
best_pipe.predict(passenger)